<a href="https://colab.research.google.com/github/parmidamardi/CDM_TA_5/blob/main/CDM_TA_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Section 1

In [ ]:
#step 1.1

In [7]:
import pandas as pd

X = pd.read_csv(
    "https://raw.githubusercontent.com/parmidamardi/CDM_TA_5/refs/heads/main/secom.data",
    sep=" ",
    header=None
)

y = pd.read_csv(
    "https://raw.githubusercontent.com/parmidamardi/CDM_TA_5/refs/heads/main/secom_labels.data",
    header=None
)

In [8]:
print(X.shape)
print(y.shape)


(1567, 590)
(1567, 1)


In [9]:
from sklearn.feature_selection import VarianceThreshold

vt = VarianceThreshold(threshold=0.0)
X_no_const = vt.fit_transform(X)


In [10]:
print("Original features:", X.shape[1])
print("Remaining features:", X_no_const.shape[1])
print("Removed zero-variance features:", X.shape[1] - X_no_const.shape[1])


Original features: 590
Remaining features: 474
Removed zero-variance features: 116


In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X_no_const)


In [12]:
print("Remaining NaNs:", np.isnan(X_imputed).sum())


Remaining NaNs: 0


In [13]:
#step 1.2

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)


In [15]:
import numpy as np

print(np.mean(X_scaled, axis=0)[:5])
print(np.std(X_scaled, axis=0)[:5])

[ 1.62332035e-15 -5.07372631e-15 -5.71336214e-15 -1.26963603e-16
  0.00000000e+00]
[1. 1. 1. 1. 1.]


In [16]:
#Section 2

In [17]:
#step 2.1

In [54]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
import numpy as np
import time


y_vec = y.values.ravel()

start_time = time.time()

mi_scores = mutual_info_classif(
    X_scaled,
    y_vec,
    random_state=42
)

end_time = time.time()
mi_time = end_time - start_time
print(f"MI execution time: {mi_time:.4f} seconds")


selected_columns = X.columns[vt.get_support()]

mi_series = pd.Series(
    mi_scores,
    index=selected_columns
)

mi_series_sorted = mi_series.sort_values(ascending=False)



MI execution time: 44.9822 seconds


In [55]:
mi_top20 = mi_series_sorted.head(20)
print(mi_top20)

249    3.041678
521    2.884178
387    2.805427
342    2.419387
114    2.411677
347    2.336227
492    2.329185
109    2.134876
478    2.101210
209    2.031040
111    1.968667
358    1.943389
220    1.801318
74     1.794580
293    1.739683
245    1.718231
85     1.692480
110    1.664892
124    1.653061
586    1.609216
dtype: float64


In [27]:
#step 2.2

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import time

X_scaled_df = pd.DataFrame(
    X_scaled,
    columns=X.columns[vt.get_support()]
)

rf = RandomForestClassifier(
    n_estimators=30,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

rfe = RFE(
    estimator=rf,
    n_features_to_select=20,
    step=20
)


In [59]:
start_time = time.time()
rfe.fit(X_scaled_df, y_vec)
end_time = time.time()

print(f"RFE execution time: {end_time - start_time:.2f} seconds")


RFE execution time: 156.39 seconds


In [61]:
#Section 3

In [70]:
#step 3.1 and 3.2 and 3.3

In [71]:
start_time = time.perf_counter()

U, S, VT = np.linalg.svd(X_scaled, full_matrices=False)

k = 20
sigma_squared = S[:k] ** 2
V_k = VT[:k, :]

svd_scores = np.sum(
    sigma_squared[:, np.newaxis] * np.abs(V_k),
    axis=0
)

svd_scores_series = pd.Series(
    svd_scores,
    index=X_scaled_df.columns,
    name="SVD_Score"
).sort_values(ascending=False)

top_20_svd_features = svd_scores_series.head(20)

end_time = time.perf_counter()

svd_time = end_time - start_time

print("Top 20 SVD features:")
print(top_20_svd_features)
print(f"\nSVD Feature Selection Time: {svd_time:.4f} seconds")


Top 20 SVD features:
209    14723.591277
478    14723.591277
347    14723.591277
206    14723.591277
342    14723.591277
74     14723.591277
62     14714.025063
303    13726.571557
338    13725.257912
202    13724.940301
168    13413.364582
267    13276.941610
405    13251.482191
440    13212.823411
539    13179.863070
339    13122.782942
336    13093.166187
46     13054.120095
406    12932.253017
304    12916.250949
Name: SVD_Score, dtype: float64

SVD Feature Selection Time: 0.4749 seconds
